# Análise Exploratória e Pré-processamento de Dados

## Introdução
Este notebook tem como objetivo realizar a **Análise Exploratória de Dados (EDA)** e preparar o dataset de solubilidade para a regressão linear que será feita posteriormente. O pipeline de trabalho consiste em:

1.  **Carregamento e Configuração do Ambiente;**
2.  **Separação de Variáveis;**
3.  **Estatísticas Descritivas e Assimetria (Skewness);**
4.  **Pré-processamento;**
5.  **Análise de Correlações;**
6.  **Visualização de Linearidade;**
7. **Diagnóstico de variância via screeplot;**
8. **Filtragem dos preditores.**


Ao final, os dados processados serão salvos para garantir que os notebooks de modelagem utilizem exatamente a mesma base tratada.

In [3]:
import pandas as pd
import os
import sys
import statsmodels
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.decomposition import PCA
from pathlib import Path

# Configuração visual
sns.set_style("ticks")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Carregamento e Configuração do Ambiente

Nessa parte, definimos os caminhos relativos para garantir que o código funcione em qualquer máquina, desde que a estrutura de diretórios seja mantida. Também carregamos os arquivos `.txt` contendo as features (`X`) e os targets (`y`) para treino e teste.

In [4]:
# =============================================================================
# CARREGAMENTO E DADOS
# =============================================================================
PROJETO_ROOT = Path.cwd().parent  # sobe de notebooks/ para HW2/


# adicionar src/ ao sys.path
SRC_PATH = PROJETO_ROOT / "src"
if SRC_PATH.exists():
    sys.path.append(str(SRC_PATH))
    print(f"src/ adicionado ao path: {SRC_PATH}")

# carregar os dados da pasta solubility
DADOS_PATH = PROJETO_ROOT / "solubility"

# verificando se os arquivos existem
arquivos_esperados = ["solTrainX.txt", "solTrainY.txt", "solTestX.txt", "solTestY.txt"]
for arquivo in arquivos_esperados:
    caminho = DADOS_PATH / arquivo
    if caminho.exists():
        print(f"{arquivo} encontrado")
    else:
        print(f"{arquivo} NÃO encontrado em {caminho}")

# carregando por fim
X_train_raw = pd.read_csv(DADOS_PATH / "solTrainX.txt", sep='\t')
y_train = pd.read_csv(DADOS_PATH / "solTrainY.txt", sep='\t').values.ravel()
X_test_raw  = pd.read_csv(DADOS_PATH / "solTestX.txt", sep='\t')
y_test  = pd.read_csv(DADOS_PATH / "solTestY.txt", sep='\t').values.ravel()

print(f"\nDados carregados com sucesso!")
print(f"X_train: {X_train_raw.shape}, X_test: {X_test_raw.shape}")

solTrainX.txt NÃO encontrado em /solubility/solTrainX.txt
solTrainY.txt NÃO encontrado em /solubility/solTrainY.txt
solTestX.txt NÃO encontrado em /solubility/solTestX.txt
solTestY.txt NÃO encontrado em /solubility/solTestY.txt


FileNotFoundError: [Errno 2] No such file or directory: '/solubility/solTrainX.txt'

## 2. Separação de Variáveis

O dataset contém dois tipos distintos de variáveis:
1.  **Binárias (Fingerprints):** Indicam a presença (1) ou ausência (0) de subestruturas moleculares. Elas têm apenas 2 valores únicos. Um exemplo desse tipo de variável pode ser: "Possui um anel benzênico?". Visto que essas variáveis são categóricas/indicadoras, transformá-las acabaria com seu significado lógico.

2.  **Contínuas:** Representam propriedades físico-químicas como peso molecular, número de átomos, área de superfície, etc.

Essa separação é essencial porque aplicaremos transformações (como Yeo-Johnson) **apenas** nas variáveis contínuas.

In [ ]:
# Separar Binárias e Contínuas
cols_binarias = [col for col in X_train_raw.columns if X_train_raw[col].nunique() <= 2]
cols_continuas = [col for col in X_train_raw.columns if col not in cols_binarias]

# 1. Exibir tipos de variáveis (binárias)
print(f"\nNúmero de variáveis binárias: {len(cols_binarias)}")
if len(cols_binarias) > 0:
    print(f"Variáveis binárias: {cols_binarias}")

# 2. Exibir tipos de variáveis (contínuas)
print(f"\nNúmero de variáveis contínuas: {len(cols_continuas)}")
if len(cols_continuas) > 0:
    print(f"Variáveis contínuas: {cols_continuas}")

## 3. Estatísticas Descritivas e Assimetria (Skewness)

Antes de modelar, precisamos entender a distribuição dos dados. A tabela abaixo fornece métricas como média, desvio padrão e quartis para as variáveis contínuas.

Além disso, calculamos o **skewness (assimetria)**. Modelos lineares (como OLS e Ridge) beneficiam-se de variáveis com distribuição próxima da normal (simétrica). Valores absolutos de skewness altos indicam a necessidade de transformação.

In [ ]:
# =============================================================================
# EDA: VISUALIZAÇÃO INICIAL DOS DADOS
# =============================================================================

print("\n>>> ANÁLISE DE ESTATÍSTICAS DESCRITIVAS E SKEWNESS")

# 3. Gerar tabela de estatísticas descritivas para variáveis contínuas
print("\nEstatísticas Descritivas para Variáveis Contínuas (X_train_raw):\n")
descriptive_stats = X_train_raw[cols_continuas].describe()
print(descriptive_stats.to_markdown(numalign="left", stralign="left"))

print("\n>>> ANALISANDO SKEWNESS DAS VARIÁVEIS CONTÍNUAS...")

# 1. Calcular o skewness (assimetria) de todas as variáveis contínuas em X_train_raw
skewness_values = X_train_raw[cols_continuas].skew()

# 2. Ordenar os valores de skewness
skewness_sorted = skewness_values.sort_values(ascending=False)

print("Skewness calculado para variáveis contínuas.")

print("\nTop 5 Variáveis Contínuas com Maior Skewness Positivo:")
print(skewness_sorted.head(5).to_markdown(numalign="left", stralign="left"))

print("\nTop 5 Variáveis Contínuas com Maior Skewness Negativo:")
print(skewness_sorted.tail(5).to_markdown(numalign="left", stralign="left"))

Como se pode ver, há vários preditores com valor absoluto da skewness acima de 1 e alguns chegando a passar de 3. Isso indica a necessidade de aplicar o pré-processamento para corrigir a assimetria e colocar os dados na mesma escala.

## 4. Pré-processamento: Transformação Yeo-Johnson

Para corrigir a assimetria identificada acima, aplicamos a transformação **Yeo-Johnson**. Ela é similar ao Box-Cox, que foi aplicado no livro-texto, mas suporta valores zero e negativos. A transformação **Yeo-Johnson** busca o parâmetro $\lambda$ que minimize a assimetria, aproximando a distribuição de uma Gaussiana (Normal).

Além disso, após a utilização do **Yeo-Johnson**, também realizamos a padronização dos dados, centralizando a média em 0 e o desvio padrão em 1.

Vale ressaltar que o (`.fit`) é aplicado apenas no conjunto de treino. Uma vez tendo achado todos os parâmetros, utilizamos o (`.transform`) no conjunto de teste. Isso garante que nenhuma informação do teste "vaze" para o treinamento.

In [ ]:
# =============================================================================
# PRÉ-PROCESSAMENTO (TRANSFORMAÇÃO)
# =============================================================================

print("\n>>> PRÉ-PROCESSAMENTO (YEO-JOHNSON)")

pt = PowerTransformer(method='yeo-johnson', standardize=True)

X_train_trans = X_train_raw.copy()
X_test_trans = X_test_raw.copy()

if len(cols_continuas) > 0:
    # Ajusta o transformador APENAS no treino para evitar data leakage
    X_train_trans[cols_continuas] = pt.fit_transform(X_train_raw[cols_continuas])
    # Aplica a mesma transformação no teste
    X_test_trans[cols_continuas] = pt.transform(X_test_raw[cols_continuas])

print("Variáveis contínuas transformadas e padronizadas.")

### 4.1. Verificação da Eficácia da Transformação

Após a transformação, recalculamos o skewness para verificar se ele foi reduzido.

In [ ]:
print("\n>>> ANALISANDO SKEWNESS DAS VARIÁVEIS CONTÍNUAS (DEPOIS da Transformação)...")

# Calcular skewness após a transformação
skewness_values_trans = X_train_trans[cols_continuas].skew()
skewness_sorted_trans = skewness_values_trans.sort_values(ascending=False)

print("\nTop 5 Variáveis com Maior Skewness Positivo (Pós-Transformação):")
print(skewness_sorted_trans.head(5).to_markdown(numalign="left", stralign="left"))

print("\nTop 5 Variáveis com Maior Skewness Negativo (Pós-Transformação):")
print(skewness_sorted_trans.tail(5).to_markdown(numalign="left", stralign="left"))

Pode-se ver que, apesar de ter algumas variáveis com **skewness** ainda acima de 1 e até de 2, os valores em geral diminuiram de forma significativa e a maioria agora possui **skewness** próximo de 0. Com isso, os dados estão mais preparados para realização da regressão.

Agora, vamos analisar o comportamento dos preditores com a variável alvo em alguns aspectos.

## 5. Análise de Correlações

Nessa célula, investigamos como as variáveis contínuas se relacionam com a variável alvo (`Solubility`). Calculamos a correlação de Pearson e exibimos as variáveis mais correlacionadas (tanto positiva quanto negativamente).

A correlação de Pearson mede a força e a direção da relação linear entre duas variáveis, variando de -1 a 1.

In [ ]:
# =============================================================================
# EDA: CORRELAÇÕES
# =============================================================================
print("\n>>> ANÁLISE DE CORRELAÇÕES (Parte 0)...")

# 1. Matriz de Correlação entre Preditores

# Nesse caso, pegamos só as contínuas para não deixar o mapa de calor muito grande
corr_matrix = X_train_trans[cols_continuas].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, square=True, cbar_kws={"shrink": .5})
plt.title("Matriz de Correlação (Preditores Contínuos)")
plt.show()

# 2. Correlação Linear Individual (Preditores vs Outcome)
y_train_series = pd.Series(y_train, index=X_train_trans.index)

correlations_y = X_train_trans.corrwith(y_train_series).sort_values(ascending=False)


print("\nTop 5 Correlações Positivas com Solubilidade:")
print(correlations_y.head(5))
print("\nTop 5 Correlações Negativas com Solubilidade:")
print(correlations_y.tail(5))


O mapa de calor acima mostra a correlação entre os preditores. Regiões vermelhas ou azuis escuras fora da diagonal principal indicam que duas variáveis explicam praticamente a mesma coisa. Isso é problemático para modelos lineares, pois infla a variância dos coeficientes, tornando o modelo instável. Essa multicolinearidade será tratada em breve.

## 6. Visualização de Linearidade

Uma vez tendo visto quais variáveis mais/menos se correlacionam com a variável alvo (`Solubility`), vamos agora visualizar também como é comportamento de cada variável com a variável alvo utilizando scatter plots com uma linha de suavização. Dessa forma, será possível verificar a linearidade entre os preditores e o alvo, podendo assim, identificar possíveis problemas para a aplicação da regressão, uma vez que o modelo linear falhará em capturar relações que não são lineares, resultando em erros sistemáticos.


In [ ]:
# =============================================================================
# VISUALIZAÇÃO DE LINEARIDADE
# =============================================================================

print("\n>>> GERANDO SCATTER PLOTS (Predictor vs Outcome)")

# Vamos pegar as variáveis contínuas transformadas

num_vars = len(cols_continuas)
cols = 4
rows = (num_vars // cols) + 1

fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
axes = axes.flatten()

for i, col in enumerate(cols_continuas):
    # Plot scatter com linha de suavização (lowess)
    sns.regplot(x=X_train_trans[col], y=y_train, ax=axes[i],
                lowess=True, # Isso faz a linha curva suave (não linear)
                scatter_kws={'alpha': 0.3, 's': 10, 'color': 'gray'},
                line_kws={'color': 'red'})
    axes[i].set_title(col)
    axes[i].set_xlabel("Transformed Value")
    axes[i].set_ylabel("Solubility")

# Remove eixos vazios se houver
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()
print("Comentário: Observe se a linha vermelha é reta (linear) ou curva (não-linear).")

Apesar de alguns preditores possuirem relações não-lineares com a variável alvo, o comportamento no geral é mais linear.

## 7. Diagnóstico de variância via screeplot

Assim como nas duas células anteriores, mais uma vez essa célula serve com o intuito de analisar o comportamento dos preditores no dataset, sem alterá-lo. Dessa vez, o objetivo é visualizar o Scree Plot dos preditores, a partir da PCA, e entender a partipação de cada variável na informação total dos dados, podendo assim descobrir quantos componentes principais seriam suficientes para representar a maior parte da informação original dos dados ao mesmo tempo que ajuda a entender a redundância da informação. Tal informação também contribui para a redução a dimensionalidade e a diminuição da multicolineariade, que é péssima para a realização da regressão linear, principalmente para aquela feita por Mínimos Quadrados Ordinários (OLS), uma vez que torna a matriz $X^T X$ quase singular (não invertível).



In [ ]:
# =============================================================================
# 3.4. DIAGNÓSTICO DE VARIÂNCIA VIA PCA - SCREE PLOT
# =============================================================================
print("\n>>> GERANDO SCREE PLOT (PCA)...")

# Para a visualização da PCA (e apenas para ela), precisamos
# colocar tudo na mesma escala para fazer o gráfico
# Isso não afeta os dados X_train_trans usados na regressão depois.

# 1. Cria uma cópia temporária para não estragar o dataset principal
X_pca_temp = X_train_trans.copy()

# 2. Padroniza tudo (Binárias + Contínuas) apenas para o PCA
scaler_global = StandardScaler()
X_pca_scaled = scaler_global.fit_transform(X_pca_temp)

# 3. Roda o PCA
pca = PCA()
pca.fit(X_pca_scaled)

# 4. Plota
explained_var = pca.explained_variance_ratio_ * 100
components = np.arange(1, len(explained_var) + 1)

print(f"Variância explicada pelo 1º Componente: {explained_var[0]:.2f}%")

plt.figure(figsize=(10, 6))
plt.plot(components, explained_var, 'b-', linewidth=1.5)
plt.title("Scree Plot")
plt.xlabel("Componente Principal")
plt.ylabel("Variância Explicada (%)")
plt.xlim(0, 200)
plt.ylim(0, 25)
plt.grid(True, alpha=0.3)
plt.show()

Com o scree plot, foi possível ver que a variável que mais possui informação, explica menos de 13% da variância total. Além disso, observa-se que a partir de uma determinada quantidade de variáveis, praticamente toda a variância foi explicada, o que confirma que há redundância (multicolinearidade) nos dados originais.

## 8. Filtragem dos preditores

Uma vez tendo analisado as correlações do preditores com a variável alvo, a linearidade e variância com scree plot, vamos combater a multicolinearidade removendo qualquer preditor que possua correlação maior que 0.9 com outro preditor.

Em outras palavras, definimos um limiar de corte (*threshold*) de **0.9**. O algoritmo percorre a matriz de correlação e, ao encontrar um par de variáveis com $|r| > 0.9$, remove uma delas. Isso simplifica o modelo, mantendo a maior parte da informação única.

No livro-texto, foi utilizada uma função em R que possui uma maneira um pouco diferente daquela feita aqui, podendo causar uma pequena diferença na quantidade de variáveis removidas.



In [ ]:
# =============================================================================
# FILTRAGEM DE PREDIRORES (Alta Colinearidade)
# =============================================================================

print("\n>>> REMOVENDO ALTA CORRELAÇÃO (>0.9)...")

def identificar_correlacoes(df, threshold=0.9):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return to_drop

cols_drop = identificar_correlacoes(X_train_trans, 0.9)
X_train_filtered = X_train_trans.drop(columns=cols_drop)
X_test_filtered = X_test_trans.drop(columns=cols_drop)

print(f"Preditores iniciais: {X_train_trans.shape[1]}")
print(f"Preditores removidos: {len(cols_drop)}")
print(f"Preditores finais: {X_train_filtered.shape[1]}")


## 9. Persistência dos Dados

Ao final disso tudo, possuimos o dataset com os dados processados. Sendo assim, ele está pronto para a aplicação do próximo passo, que no caso é a OLS. Isso será visto no arquivo (`analise_OLS.ipynb`).

Salvamos os dados transformados em um arquivo `pickle` para que possam ser carregados rapidamente nos notebooks de modelagem, garantindo consistência em todo o projeto.

In [ ]:
import pickle

with open("dados_preprocessados.pkl", "wb") as f:
    pickle.dump(
        (X_train_filtered, X_test_filtered, y_train, y_test),
        f
    )

print("Dados pré-processados salvos com sucesso!")
